In [30]:
import cv2
import numpy as np

def outline_object(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours (use the same size as the original image)
    contour_img = np.zeros_like(img)

    # Draw contours on the empty canvas
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)  # Draw contours in green

    # Show the outlined object
    cv2.imshow('Outlined Object', contour_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Replace 'path_to_your_image.png' with the path to your image
outline_object('AF_left_20000.png')


In [31]:
import cv2
import numpy as np

def outline_object_with_square_box(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Draw the square box around the bounding rectangle of all the green contours
    cv2.rectangle(overlay_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Draw the outlined object on the canvas
    cv2.drawContours(overlay_img, contours, -1, (0, 255, 0), 2)  # Draw contours in green

    # Show the outlined object with the square box
    cv2.imshow('Outlined Object with Square Box', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Replace 'path_to_your_image.png' with the path to your image
outline_object_with_square_box('AF_left_20000.png')


Vertical segmentation

In [15]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the width of the square box
    width_segments = np.linspace(x, x + w, num=26, dtype=int)

    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, and space for each segment
    for idx in range(len(width_segments) - 1):
        segment_x_start = width_segments[idx]
        segment_x_end = width_segments[idx + 1]
        segment_width = segment_x_end - segment_x_start  # Width of the segment

        start_point = (segment_x_start, y)
        end_point = (segment_x_end, y + h)
        mid_point = ((segment_x_start + segment_x_end) // 2, y + h // 2)
        segment_coordinates.append((start_point, mid_point, end_point))

        # Calculate segment space
        if idx > 0:
            space = segment_x_start - width_segments[idx - 1]
        else:
            space = 0  # No space for the first segment
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Vertical_coordinate.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('whitematter-images/T_PREC_right_00000.png')


Horizontal Segmentation

In [18]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the height of the square box
    height_segments = np.linspace(y, y + h, num=26, dtype=int)

    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, end coordinates, and space for each segment
    for idx in range(len(height_segments) - 1):
        segment_y_start = height_segments[idx]
        segment_y_end = height_segments[idx + 1]

        start_point = (x, segment_y_start)
        end_point = (x + w, segment_y_end)
        mid_point = (x + w // 2, (segment_y_start + segment_y_end) // 2)
        
        # Calculate space between segments
        space = segment_y_end - segment_y_start
        
        segment_coordinates.append((start_point, mid_point, end_point))
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Horizontal_coordinates.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('whitematter-images/CG_left_20000.png')


ADD ON

Manually-Horizontal

In [34]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the height of the square box
    height_segments = np.linspace(y, y + h, num=26, dtype=int)

    # Increase space between segments
    increased_height_segments = np.linspace(y, y + h, num=26, dtype=int)
    increased_height_segments[1:-1] += 5  # Increase space by 5 pixels for each segment except the first and last
    increased_height_segments[0] -= 15  # Increase space for the first segment
    increased_height_segments[-1] += 5  # Increase space for the last segment

    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, end coordinates, and space for each segment
    for idx in range(len(increased_height_segments) - 1):
        segment_y = (increased_height_segments[idx] + increased_height_segments[idx + 1]) // 2  # Center point between start and end Y coordinates
        
        start_point = (x, segment_y)
        end_point = (x + w, segment_y)
        mid_point = (x + w // 2, segment_y)
        
        # Calculate space between segments
        space = increased_height_segments[idx + 1] - increased_height_segments[idx]
        
        segment_coordinates.append((start_point, mid_point, end_point))
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Horizontal-manually_coordinates.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('AF_left_20000.png')


Dynamically-horizontal

In [35]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the contour height
    segment_heights = np.linspace(y, y + h, num=26, dtype=int)

    # Calculate segment spaces based on contour height
    contour_height = h
    avg_space = contour_height / 25

    # Adjust segment heights to cover the entire contour area
    for i in range(1, len(segment_heights) - 1):
        space = segment_heights[i + 1] - segment_heights[i]
        if space < avg_space:
            diff = avg_space - space
            segment_heights[i + 1:] = (segment_heights[i + 1:] + diff).astype(int)


    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, end coordinates, and space for each segment
    for idx in range(len(segment_heights) - 1):
        segment_y = (segment_heights[idx] + segment_heights[idx + 1]) // 2  # Center point between start and end Y coordinates
        
        start_point = (x, segment_y)
        end_point = (x + w, segment_y)
        mid_point = (x + w // 2, segment_y)
        
        # Calculate space between segments
        space = segment_heights[idx + 1] - segment_heights[idx]
        
        segment_coordinates.append((start_point, mid_point, end_point))
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Horizontal-dynamically_coordinates.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('ST_PREF_left_00000.png')


Vertically-manually

In [36]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the width of the square box
    width_segments = np.linspace(x, x + w, num=26, dtype=int)

    # Increase space between segments
    increased_width_segments = np.linspace(x, x + w, num=26, dtype=int)
    increased_width_segments[1:-1] += 5  # Increase space by 5 pixels for each segment except the first and last
    increased_width_segments[0] -= 15  # Increase space for the first segment
    increased_width_segments[-1] += 5  # Increase space for the last segment

    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, end coordinates, and space for each segment
    for idx in range(len(increased_width_segments) - 1):
        segment_x = (increased_width_segments[idx] + increased_width_segments[idx + 1]) // 2  # Center point between start and end X coordinates
        
        start_point = (segment_x, y)
        end_point = (segment_x, y + h)
        mid_point = (segment_x, y + h // 2)
        
        # Calculate space between segments
        space = increased_width_segments[idx + 1] - increased_width_segments[idx]
        
        segment_coordinates.append((start_point, mid_point, end_point))
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Vertical-manually_coordinates.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('AF_left_20000.png')


Vertically-dynamic

In [37]:
import cv2
import numpy as np
import csv

def save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, filename):
    # Write coordinates and spaces to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Segment', 'Start X', 'Start Y', 'Mid X', 'Mid Y', 'End X', 'End Y', 'Space'])
        for idx, ((start_point, mid_point, end_point), space) in enumerate(zip(segment_coordinates, segment_spaces), start=1):
            writer.writerow([idx, start_point[0], start_point[1], mid_point[0], mid_point[1], end_point[0], end_point[1], space])

def visualize_start_end_coordinates(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection or suitable filters
    edges = cv2.Canny(gray, 30, 100)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty canvas to draw the contours and square box
    overlay_img = img.copy()

    # Combine all contours into one to find the bounding rectangle
    combined_contours = np.concatenate(contours)

    # Get the bounding rectangle of the combined contours
    x, y, w, h = cv2.boundingRect(combined_contours)

    # Calculate segment points along the width of the square box
    width_segments = np.linspace(x, x + w, num=26, dtype=int)

    # Calculate segment spaces based on contour width
    contour_width = w
    avg_space = contour_width / 25

    # Adjust segment widths to cover the entire contour area
    for i in range(1, len(width_segments) - 1):
        space = width_segments[i + 1] - width_segments[i]
        if space < avg_space:
            diff = avg_space - space
            width_segments[i + 1:] = (width_segments[i + 1:] + diff).astype(int)

    # List to store segment coordinates and spaces
    segment_coordinates = []
    segment_spaces = []

    # Calculate and store start, mid, end coordinates, and space for each segment
    for idx in range(len(width_segments) - 1):
        segment_x = (width_segments[idx] + width_segments[idx + 1]) // 2  # Center point between start and end X coordinates
        
        start_point = (segment_x, y)
        end_point = (segment_x, y + h)
        mid_point = (segment_x, y + h // 2)
        
        # Calculate space between segments
        space = width_segments[idx + 1] - width_segments[idx]
        
        segment_coordinates.append((start_point, mid_point, end_point))
        segment_spaces.append(space)

        # Draw the segments and midpoints on the overlay image
        cv2.rectangle(overlay_img, start_point, end_point, (0, 0, 255), 2)
        cv2.circle(overlay_img, mid_point, 5, (0, 255, 0), -1)

    # Display start and end coordinates of the first segment on the image
    cv2.putText(overlay_img, f"Start: {segment_coordinates[0][0]}", segment_coordinates[0][0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay_img, f"End: {segment_coordinates[0][2]}", segment_coordinates[0][2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save segment coordinates and spaces to a CSV file
    save_segment_coordinates_to_csv(segment_coordinates, segment_spaces, 'tract-Vertical-dynamically_coordinates.csv')

    # Show the image with segments, midpoints, and coordinates
    cv2.imshow('Start and End Coordinates', overlay_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Visualize the start and end coordinates of the first segment
visualize_start_end_coordinates('ST_PREF_left_00000.png')
